<a href="https://colab.research.google.com/github/LennartReddm/DashboardPADC/blob/main/Interactive_Dashboard_Production_Attacker_Defender_Contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Interactive Dashboard for Mixed Strategy Nash Equilibrium in the PAD-C

Please note that in order for the dashboard to work fully, you will need to load the "Packages and Functions"-cell further down first and then re-run the "Display interactive dashboard"-cell immediately below.

In [3]:
# Display interactive dashboard with a maximum endowment e of 30 MU (monetary units) and a maximum production pay-off k of 40 MU.
display_interactive_dashboard(30, 40)

<center><h3>Mixed Strategy Nash Equilibrium for the Production Attacker-Defender Contest (PAD-C)</h3></center>

AppLayout(children=(VBox(children=(HBox(children=(Label(value='Model parameters:'),)), HBox(children=(IntSlide…

**Note**: The plot depicts the probability distribution with which each conflict investment strategy from 0 to e should be chosen by the attacker (red) and defender (blue) in the PAD-C under Nash equilibrium, depending on the selected production threshold T and payoff "reward" k, and assuming rational selfish play and risk neutrality. Table shows average play for key variables calculated with the probability weights of the distribution.

### Packages and Functions **Load this first**

In [1]:
# PACKAGES

# Install packages
!pip install quantecon

# Load packages
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import quantecon.game_theory as gt

# Local application imports
from IPython.display import display, Markdown, clear_output


# FUNCTIONS

## MAIN FUNCTIONS TO CREATE INTERACTIVE DASHBOARD

# Function that renders interactive dashboard using ipywidgets, specifically AppLayout
def display_interactive_dashboard(eMax, kMax):
    style = {'description_width': 'initial'}
    T = widgets.IntSlider(min=0, max=eMax, step=1, value=0, description='Threshold T:', style=style)
    k = widgets.IntSlider(min=0, max=kMax, step=1, value=0, description='Payoff k:', style=style)
    e = widgets.IntSlider(min=0, max=eMax, step=1, value=20, description='Endowment e:', style=style)
    button = widgets.Button(description="Update")

    plotOutput = widgets.Output() # Use Output widget instead of interactive_output
    tableOutput = widgets.Output()

    def on_button_clicked(b=None):  # Make the button argument optional
        T_value = T.value
        k_value = k.value
        e_value = e.value
        with plotOutput:
            clear_output(wait=True) # Clear previous plots
            PADCEquilibPlotOutput(T_value, k_value, e_value)
            plt.show()
        with tableOutput:
            clear_output(wait=True) # Clear previous tables
            PADCEquilibTableOutput(T_value, k_value, e_value)

    button.on_click(on_button_clicked)

    # Call the function once to display the plot and table with default values
    on_button_clicked()

    # Create slider box with title
    slider_title = widgets.Label('Model parameters:')
    title_box = widgets.HBox([slider_title])
    sliders_box = widgets.HBox([T, k, e, button])
    sliderVBox = widgets.VBox([title_box, sliders_box])

    # Create a label for the dashboard title
    dashboard_title = widgets.Label('Mixed Strategy Nash Equilibrium for the Production Attacker-Defender Contest (PAD-C)', layout=widgets.Layout(align_items='center'))
    display(Markdown("<center><h3>Mixed Strategy Nash Equilibrium for the Production Attacker-Defender Contest (PAD-C)</h3></center>"))
    # Use AppLayout to display the elements together.
    dashboard = widgets.AppLayout(header=None,
                                  left_sidebar=plotOutput,
                                  center=tableOutput,
                                  right_sidebar=None,
                                  footer=sliderVBox,
                                  pane_widths=[11, 4.5, 1],
                                  grid_gap="0px")

    return dashboard

# Function to create a dynamic plot with equilibrium predictions using the same
# production pay-off k and treshold T for attacker and defender.
def PADCEquilibPlotOutput(T, k, e):

  # Create tables with equilibrium strategies
  PADCEquilibArr = PADCequilib(e, e, T, T, k, k)
  PADCEquilibTableAtt = pd.DataFrame({'Attacker': PADCEquilibArr[0]})
  PADCEquilibTableDef = pd.DataFrame({'Defender': PADCEquilibArr[1]})

  # Create plot with equilibrium strategies
  fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,6))  # 1 row, 2 columns

  # Specify subplot for attacker
  PADCEquilibTableAttPlt = PADCEquilibTableAtt.plot(kind='bar',color=['Red'], ylim=(0,1), width=0.8, rot=50, ax=ax1)
  PADCEquilibTableAtt = PADCEquilibTableAtt.T
  ax1.table(cellText=np.round(PADCEquilibTableAtt.values, 2), colLabels=PADCEquilibTableAtt.columns, rowLabels=['Probability'] , loc='bottom', cellLoc='center', rowLoc='center')
  ax1.get_xaxis().set_visible(False)
  table = PADCEquilibTableAttPlt.tables[0]
  table.auto_set_font_size(False)
  table.scale(1, 1.3)
  for key, cell in table.get_celld().items():
      cell.set_edgecolor('lightgrey')
  #removing top and right borders
  ax1.spines['top'].set_visible(False)
  ax1.spines['right'].set_visible(False)
  ax1.get_legend().remove()
  #Change color of grid lines and ticks
  ax1.spines['bottom'].set_color('lightgrey')
  ax1.spines['left'].set_color('lightgrey')
  ax1.tick_params(axis='y', color='lightgrey')

  # Specify subplot for defender
  PADCEquilibTableDefPlt = PADCEquilibTableDef.plot(kind='bar',color=['Blue'], ylim=(0,1), width=0.8, rot=50, ax=ax2)
  PADCEquilibTableDef = PADCEquilibTableDef.T
  ax2.table(cellText=np.round(PADCEquilibTableDef.values, 2), colLabels=PADCEquilibTableAtt.columns , loc='bottom', cellLoc='center' )
  ax2.get_xaxis().set_visible(False)
  ax2.get_yaxis().set_visible(False)
  table = PADCEquilibTableDefPlt.tables[0]
  table.auto_set_font_size(False)
  table.scale(1, 1.3)
  for key, cell in table.get_celld().items():
      cell.set_edgecolor('lightgrey')
  #removing top and right borders
  ax2.spines['top'].set_visible(False)
  ax2.spines['right'].set_visible(False)
  ax2.spines['left'].set_visible(False)
  ax2.get_legend().remove()
  #Change color of grid lines and ticks
  ax2.spines['bottom'].set_color('lightgrey')
  ax2.spines['left'].set_color('lightgrey')

  display(Markdown("<center><H4>Mixed Strategy Probability Distribution for Conflict Investments</H4></center>"))
  plt.tight_layout()
  plt

# Function to create a dynamic table with weighted average equilibrium predictions based on T, k, and e.
def PADCEquilibTableOutput(T, k, e):

  # Create table with equilibrium strategies
  PADCEquilibArr = PADCequilib(e, e, T, T, k, k)
  tblOptimalProdPO = tableOptimalProdPO(T, k, e)
  PADCEquilibTable = pd.DataFrame({'Conflict Strategy': list(range(0, e+1)),
                                   'Optimal Keep': tblOptimalProdPO['Optimal Keep for CI level'],
                                   'Optimal P': tblOptimalProdPO['Optimal p for CI level'],
                                   'Att Probability': PADCEquilibArr[0],
                                   'Def Probability': PADCEquilibArr[1]})

  # Calculate Expected Earnings
  PODmat = createDefPOMat(e, e, T, k)
  POAmat = createAttPOMat(e, e, T, T, k, k)
  probPOMat = np.empty((e+1, e+1))
  for i in range(e+1):
    for j in range(e+1):
      probPOMat[i,j] = PADCEquilibArr[0][i] * PADCEquilibArr[1][j]
  expEarningsAtt = np.multiply(probPOMat, POAmat).sum()
  expEarningsDef = np.multiply(probPOMat, PODmat).sum()

  # Attacker
  # Calculate weighted average Keep
  avgKeepAtt = (PADCEquilibTable['Optimal Keep'] * PADCEquilibTable['Att Probability']).sum()
  # Calculate weighted average Production
  avgprodAtt = (PADCEquilibTable['Optimal P'] * PADCEquilibTable['Att Probability']).sum()
  # Calculate weighted average Conflict
  avgconflAtt = (PADCEquilibTable['Conflict Strategy'] * PADCEquilibTable['Att Probability']).sum()

  # Defender
  # Calculate weighted average Keep
  avgKeepDef = (PADCEquilibTable['Optimal Keep'] * PADCEquilibTable['Def Probability']).sum()
  # Calculate weighted average Production
  avgprodDef = (PADCEquilibTable['Optimal P'] * PADCEquilibTable['Def Probability']).sum()
  # Calculate weighted average Conflict
  avgconflDef = (PADCEquilibTable['Conflict Strategy'] * PADCEquilibTable['Def Probability']).sum()

  # Put aggregate variables together in one
  PADCEquilibAvgsTable = pd.DataFrame({'Variable': ['Conflict Investment', 'Production Investment', 'Keep', 'Earnings'],
                                       'Attacker': [avgconflAtt, avgprodAtt, avgKeepAtt, expEarningsAtt],
                                       'Defender': [avgconflDef, avgprodDef, avgKeepDef, expEarningsDef]})

  display(Markdown("<center><h4>Weighted Average for Key Variables</h4></center>"))
  display(PADCEquilibAvgsTable)


## PAY-OFF FUNCTIONS

# Defender conflict pay-off function for the PAD-C.
def poDef(a, d, TDef, eDef, kDef):
    if a > d:
        return 0
    else:
        return (eDef - d + lDef(d, TDef, eDef, kDef))

# Defender production function
def lDef(d, TDef, eDef, kDef):
    if ((eDef - d) >= TDef) & (kDef > TDef): # Shouldn't this be *only* if the pay-off k > T, otherwise if k ≤ T the return is negative and nothing should be invested.
        return kDef - TDef
    else:
        return 0

# Attacker conflict pay-off function for the PAD-C.
def poAtt(a, d, TAtt, eAtt, TDef, eDef, kAtt, kDef):
    if a > d:
        return (eAtt - a + lAtt(a, TAtt, eAtt, kAtt)) + (eDef - d + lDef(d, TDef, eDef, kDef))
    else:
        return (eAtt - a + lAtt(a, TAtt, eAtt, kAtt))

# Attacker production function
def lAtt(a, TAtt, eAtt, kAtt):
    if ((eAtt - a) >= TAtt) & (kAtt > TAtt):
        return kAtt - TAtt
    else:
        return 0


## OTHER FUNCTIONS

### Create pay-off matrizes

# Function to create Attacker Pay-off Matrix
def createAttPOMat(eDef, eAtt, TDef, TAtt, kDef, kAtt):
    POAmat = np.zeros((eAtt+1, eDef+1),dtype=int)
    for i in range(eAtt+1):
        for j in range(eDef+1):
            POAmat[i,j]=poAtt(i, j, TAtt, eAtt, TDef, eDef, kAtt, kDef)
    return POAmat

# Function to create Defender Pay-off Matrix
def createDefPOMat(eDef, eAtt, TDef, kDef):
    PODmat = np.zeros((eDef+1,eAtt+1),dtype=int)
    for i in range(eDef+1):
        for j in range(eAtt+1):
            PODmat[i,j]=poDef(i, j, TDef, eDef, kDef)
    return PODmat


### Calculate and return equilibrium predictions based on pay-off matrizes using Lemke-Howson algorithm

# Function that created equlibrium outputs for each T and k.
def PADCequilib(eDef, eAtt, TDef, TAtt, kDef, kAtt):
    PODmat = createDefPOMat(eDef, eAtt, TDef, kDef)
    POAmat = createAttPOMat(eDef, eAtt, TDef, TAtt, kDef, kAtt)
    POADmat = np.dstack((POAmat,PODmat))
    g_POADmat = gt.NormalFormGame(POADmat)
    equilib = gt.lemke_howson(g_POADmat)
    return equilib


### Equilibrium table

# Table calculated optimal choice between investment in production p and Keep for configuration of Threshold T and pay-off k.
# For each level of conflict investment (here called CI) there is only *one* optimal choice between investment in production p and Keep.
def tableOptimalProdPO(T, k, e):
  PADCOptimalp = pd.DataFrame({'Conflict Investment (CI) level': list(range(0, e+1))})
  PADCOptimalp['MU left'] = list(reversed(range(0, e+1)))
  PADCOptimalp['Threshold T'] = T
  PADCOptimalp['Optimal Keep for CI level'] = np.nan
  PADCOptimalp['Optimal p for CI level'] = np.nan
  PADCOptimalp['Max PO prod + keep for CI level'] = np.nan
  for index, row in PADCOptimalp.iterrows():
    i = row['MU left']
    if i >= T:
      PADCOptimalp.loc[index, 'Optimal p for CI level'] = T
      PADCOptimalp.loc[index, 'Optimal Keep for CI level'] = PADCOptimalp.loc[index, 'MU left'] - T
      PADCOptimalp.loc[index, 'Max PO prod + keep for CI level'] = k + PADCOptimalp.loc[index, 'MU left'] - T
    else:
      PADCOptimalp.loc[index, 'Optimal p for CI level'] = 0
      PADCOptimalp.loc[index, 'Optimal Keep for CI level'] = PADCOptimalp.loc[index, 'MU left']
      PADCOptimalp.loc[index, 'Max PO prod + keep for CI level'] = PADCOptimalp.loc[index, 'MU left']
  return PADCOptimalp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 3.4 MB/s eta 0:00:00
